# Практика 2

Вернемся к ~нашим баранам~ датасету с климатом в Дэли.

In [ ]:
!git clone https://github.com/ArChanDDD/TS-MCSSirius-2024.git

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('TS-MCSSirius-2024/data/DailyDelhiClimateTrain.csv')
df['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in list(df['date'])]
df.head()

In [ ]:
df['target'] = df['meantemp']
_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df['date'], df['target'])

# Стационарный ряд

Проверим наш ряд на стационарность с помошью теста Дики-Фуллера

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Выполняем тест Дики-Фуллера на стационарность
result = adfuller(df['meantemp'])

# Вывод результатов теста
print('ADF статистика:', result[0])
print('p-значение:', result[1])

Понятно что ряд нестационарный (и по графику видно), p-значние > 0,05

## Задание 1 - приведение ряда к стационарному виду - 1.5 балла
Приведите ряд к стационарному не используя библиотечных функций.
Для этого по сути нужно дифференцировать ряд - в дискретном случае (как у нас), достаточно посчитать разность соседних значений.

In [ ]:
def make_stationary(time_series):
  stationary_series = [np.nan] # заполняем первое значение пустым, потому что разность не определена
  for i in range(1, len(time_series)):
      # Вычисляем разность между текущим и предыдущим элементом ряда
      # TODO

      # Добавляем разность в стационарный ряд
      # TODO

  return pd.Series(stationary_series)

assert len(make_stationary(df['meantemp']).compare(df['meantemp'].diff()))==0

Посмотрим что получилось

In [ ]:
df['target'] = make_stationary(df['meantemp'])

_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df['date'], df['target'])

In [ ]:
# Выполняем тест Дики-Фуллера на стационарность
result = adfuller(df['target'].dropna())

# Вывод результатов теста
print('ADF статистика:', result[0])
print('p-значение:', '%.30f' % result[1])

Теперь он ну очень стационарный. Только вот мы совсем никак не учли сезонность. Предсказывать что-то на таком ряду сомнительно. Мы знаем что у нас данные по дням, а значит есть сезонность 365 дней.

In [ ]:
def make_stationary(time_series, seasonality):
  stationary_series = [np.nan] * seasonality # заполняем первое значение пустым, потому что разность не определена
  for i in range(seasonality, len(time_series)):
      # Вычисляем разность между текущим и предыдущим элементом ряда
      difference = time_series[i] - time_series[i-seasonality] # это выпиливаем
      # Добавляем разность в стационарный ряд
      stationary_series.append(difference)

  return pd.Series(stationary_series)

assert len(make_stationary(df['meantemp'],365).compare(df['meantemp'].diff(365)))==0

In [ ]:
df['target'] = df['meantemp'].diff(365)

_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df['date'], df['target'])

Hу так же намного красивее, правда? Убедимся что он все ещё стационарный.


In [ ]:
# Выполняем тест Дики-Фуллера на стационарность
result = adfuller(df['target'].dropna())

# Вывод результатов теста
print('ADF статистика:', result[0])
print('p-значение:', '%.30f' % result[1])

p-value всё ещё достаточно меньше чем 0,05.

# AR(p)

Вернемся к нашим стандартным данным и попробуем реализовать метод `AR(p)`.

Напомним, согласно этому методу:

$$
  Y_t = Y_{t-1}b_1 + Y_{t-2}b_2 + \dots + Y_{t-p}b_p
$$

Фактически, это линейная регрессия, для которой мы задаем только $p$ - максимальный лаг, на который мы смотрим.

Давайте начнем.

In [ ]:
# Выберем за таргет температуру
target = 'meantemp'

## Задание 2 - реализуйте функцию создания лага - 1.5 балла

Для этого нужно для каждого $i$ от $1$ до $p$ включительно создать добавить в массив `df_lag` список температур, начиная с $i$ элемента.

Учтите, что делать `append` можно только для списков с той же длиной, что и у других списков в этом массиве. Поэтому рекомендуем после "обрезания" списка делать еще `+ [0] * i` - так вы оставите длину списка прежней.

Например, для списка `[1,2,3,4,5,6,7,8,9]` и `i=3` мы получим список `[4,5,6,7,8,9,0,0,0]`.

p.s. вообще можете делать своим методом - важен лишь результат

In [ ]:
P = 307
df_lag = []

def create_lag(arr, i):
  # TODO

for i in range(1, P+1):
  df_lag.append(create_lag(df[target], i))

Теперь сделаем из них привычные нам `X` и `y` - для этого просто транспонируем полученные лаги, и обрежем их до p с конца - потому что $p-1$ строка как раз содержит данные, необходимые для предсказания последнего в `df[target]` элемента.

In [ ]:
X = np.array(df_lag).reshape(-1, P)[:-P]
y = np.array(df[target][P:]).reshape(-1, 1)

## Задание 3 - реализуйте линейную регрессию и обучите ее - 1.5 балла

Все как делали на прошлой практике - импортируем, создаем модель, и обучаем ее :)

In [ ]:
# TODO

Проверим вашу модель - для этого предскажем данные для `X` и нарисуем че получилось:

In [ ]:
y_pred = model.predict(X)

_, _ = plt.subplots(1,1, figsize = (20,10))
plt.plot(y)
plt.plot(y_pred)

При достаточно большом $p$ тут можно увидеть крутое совпадение. Если у вас не так - попробуйте увеличить $p$ и запустить ячейки заново.

# Проверка результата

Но не так важно, как мы предсказываем существующие данные - мы же на них и обучались!

Давайте лучше создадим валидационный датасет и проверим качество на нем!

In [ ]:
df_train = df[['date', 'meantemp']][:1300]
df_val = df[['date', 'meantemp']][1300:]

In [ ]:
plt.plot(df_train['date'], df_train['meantemp'], label='train')
plt.plot(df_val['date'], df_val['meantemp'], label='val')
_ = plt.legend()

In [ ]:
def prepare_and_train_model(p):
  df_lag = []
  for i in range(1, p+1):
    df_lag.append(create_lag(df_train['meantemp'], i))
  X = np.array(df_lag).reshape(-1, p)[:-p]
  y = np.array(df_train[target][p:]).reshape(-1, 1)

  model = LinearRegression()
  model.fit(X, y)

  return model

def smart_predict(model, p):
  df_lag = list(df_train['meantemp'][-p:])
  predicted = []
  while len(predicted) != len(df_val):
    predicted_val = model.predict([df_lag])[0,0]
    predicted.append(predicted_val)
    df_lag.append(predicted_val)
    df_lag = df_lag[1:]
  return predicted

## Задание 4 - используя функции выше, напишите код для получения предсказанных значений - 1.5 балла

In [ ]:
def pipeline(p):
  # TODO
  predicted = ...
  return predicted

In [ ]:
def plot(pred):
  plt.plot(df_train['date'], df_train['meantemp'], label='train')
  plt.plot(df_val['date'], df_val['meantemp'], label='val')
  plt.plot(df_val['date'], pred, label='pred')
  _ = plt.legend()

In [ ]:
plot(pipeline(P))

In [ ]:
def mse(y_true, y_pred):
  return np.mean((y_pred - y_true) ** 2)

## Задание 5 - Подберите такой p, чтобы MSE был ниже 30. - 4 балла

In [ ]:
# TODO